<a href="https://colab.research.google.com/github/mitraslav/modeling/blob/main/modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
# Подключаем пакет readxl: нужен для чтения Excel-файлов .xlsx
library(readxl)

# Подключаем пакет dplyr: нужен для обработки таблиц (pipe %>%, group_by, summarise, mutate)
library(dplyr)

# Читаем данные из файла Task11.xlsx в таблицу df
# Ожидаемые столбцы: id (номер испытуемого), signal (0/1), response (0/1)
df <- read_excel("Task11.xlsx")

# Считаем по каждому испытуемому экспериментальные показатели:
# H = hit-rate (доля попаданий среди сигнал-проб)
# F = false alarm rate (доля ложных тревог среди шум-проб)
rates <- df %>%
  group_by(id) %>%                                  # группируем строки по испытуемому
  summarise(
    n_sig   = sum(signal == 1),                     # сколько было проб с сигналом
    n_noise = sum(signal == 0),                     # сколько было проб без сигнала (шум)
    hits    = sum(signal == 1 & response == 1),     # число попаданий (hit)
    fas     = sum(signal == 0 & response == 1),     # число ложных тревог (false alarm)
    H       = hits / n_sig,                         # доля попаданий
    F       = fas  / n_noise                        # доля ложных тревог
  )

# Определяем z-функцию: переводит вероятность p (0..1) в z-оценку стандартной нормали
# (это обратная функция Φ^{-1}(p))
z <- function(p) qnorm(p)

# Добавляем d' (чувствительность) для каждого испытуемого:
# d' = z(H) - z(F)
# Также делаем округление d' до 1 знака
rates <- rates %>%
  mutate(
    dprime_raw = z(H) - z(F),          # точный d'
    dprime     = round(dprime_raw, 1)  # округлённый d'
  )

p_signal <- 0.25   # вероятность пробы с сигналом
p_noise  <- 0.75   # вероятность пробы с шумом

R_h      <- 3      # награда за hit (попадание)
C_fa     <- 2      # штраф за false alarm (величина штрафа, положительное число)

# Оптимальное отношение правдоподобий b (критерий по оплате):
# b = (p(шум)/p(сигнал)) * (C(fa)/R(h))
# Для наших чисел b = (0.75/0.25) * (2/3) = 2
b_opt <- (p_noise / p_signal) * (C_fa / R_h)


# Используем связь b = exp(c * d') => c = ln(b) / d'
# Затем считаем оптимальные вероятности по формулам:
# H*  = 1 - Φ(c - d')
# FA* = 1 - Φ(c)
rates <- rates %>%
  mutate(
    c_opt = log(b_opt) / dprime,        # оптимальный критерий c (используем округлённый d')

    H_opt = 1 - pnorm(c_opt - dprime),  # оптимальный hit-rate: H* = 1 - Φ(c - d')
    F_opt = 1 - pnorm(c_opt),           # оптимальный FA-rate:  FA* = 1 - Φ(c)

    # Ожидаемый заработок на одну пробу:
    # E = p(signal)*H* * R(h) - p(noise)*FA* * C(fa)
    EV_trial = p_signal * H_opt * R_h - p_noise * F_opt * C_fa,

    # Ожидаемый заработок на 1000 проб:
    EV_1000  = EV_trial * 1000
  )

# Выводим итоговую таблицу с результатами
rates


id,n_sig,n_noise,hits,fas,H,F,dprime_raw,dprime,c_opt,H_opt,F_opt,EV_trial,EV_1000
<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,480,520,339,150,0.7062500,0.2884615,1.100347,1.1,0.6301338,0.6807747,0.2643035,0.1141257,114.1257
2,520,480,388,124,0.7461538,0.2583333,1.310927,1.3,0.5331901,0.7784027,0.2969510,0.1383756,138.3756
